# 10.1

In [2]:
# 최근 영업일 데이터 뽑기

import requests as rq
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/sise/sise_deposit.nhn'
data = rq.get(url)
data_html = BeautifulSoup(data.content)
parse_day = data_html.select_one(
    'div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text   # select_one - 해당 데이터 태그 추출

print(parse_day)

  |  2025.01.08


In [3]:
import re

biz_day = re.findall('[0-9]+', parse_day)   # 숫자에 해당하는 부분 추출
biz_day = ''.join(biz_day)  # 숫자 합쳐주기

print(biz_day)

20250108


## 10.2 한국거래소 업종분류 현황 및 개별지표 크롤링

## 10.2.1

In [4]:
# 1. http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201에서 CSV 데이터를 받아오기 위한 OTP 받아오기

import requests as rq
import cloudscraper     # 보안서비스 우회 요청 처리 라이브러리
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_stk = {
    'locale':'ko-KR',
    'mktId': 'STK', # 코스피 종목
    'trdDd': biz_day,
    'money':'1',
    'csvxls_isNo':'false',
    'name':'fileDown',
    'url':'dbms/MDC/STAT/standard/MDCSTAT03901'
}
# headers = {'Referer':'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201'}   # Referer: 링크를 통해 각각의 웹사이트로 방문할 때 남는 흔적, 이 흔적이 없으면 서버가 로봇으로 인식해 정보를 주지 않음

scraper = cloudscraper.create_scraper()

response = scraper.post(gen_otp_url, params=gen_otp_stk)    # 포스트 요청 - cloudscraper 사용하여 요청 우회

otp_stk = response.text

print(otp_stk)

c6G6oTqsbh4JUdDI8Q6T9cgUF6fLHO7VwODXGbOfE6MRtSksuLS7Bnxpl86F7dAOkunw9BBwugQaSjGAcH15eexkRSByVIw6IW8vUo0L8FstBgM+EFJCxYg3zco1gIgRZqIo4cIzoURnTI8+MmkJ4v/rk8yudrOQ53ef0cNipdpGgCxt8eSPYqQ0+GMW+F/bzXt/wiFT3lY4zZLV84P+nFYo9mrWvGCVTr7yVbjg2wtc9glyj0xDQ2lZrfrchZWN


In [5]:
# 위의 과정을 거처 생성된 OTP를 제출 시, 원하는 데이터 다운 가능

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_sector_stk = scraper.post(down_url, params={'code': otp_stk})              # cloudscraper 사용
sector_stk = pd.read_csv(BytesIO(down_sector_stk.content), encoding='EUC-KR')   # BytesIO : 받은 데이터의 content 부분을 바이너리 스트림으로 만든 후 read_csv 함수를 통해 데이터 읽어오기

sector_stk.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,244920,A Plus Asset,KOSPI,Other Financial Companies,4260,-135,-3.07,96308772180
1,140910,A REIT,KOSPI,Real Estate,3130,0,0.00,13969290160
2,015260,A&P,KOSPI,Electrical Equipment & Electronics,613,6,0.99,27656656022
3,078520,ABLE C&C,KOSPI,Chemicals,7250,-60,-0.82,188602667250
4,018250,AEKYUNG INDUSTRIAL,KOSPI,Chemicals,12930,0,0.00,341480459550


In [6]:
# 코스닥 시작 데이터도 동일 과정을 통해 다운

gen_otp_ksq = {
    'locale':'ko-KR',
    'mktId': 'KSQ', # 코스피 종목
    'trdDd': biz_day,
    'money':'1',
    'csvxls_isNo':'false',
    'name':'fileDown',
    'url':'dbms/MDC/STAT/standard/MDCSTAT03901'
}

otp_ksq = scraper.post(gen_otp_url, params=gen_otp_ksq).text

down_sector_ksq = scraper.post(down_url, params={'code': otp_ksq})
sector_ksq = pd.read_csv(BytesIO(down_sector_ksq.content), encoding='EUC-KR')

sector_ksq.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,366030,09WOMEN,KOSDAQ,Textiles & Apparel,3820,25,0.66,86537707000
1,159580,0TO7,KOSDAQ,Distribution,5820,20,0.34,116589941520
2,177900,3ALogics,KOSDAQ,Electrical Equipment & Electronics,7410,-120,-1.59,68815188000
3,060310,3S KOREA,KOSDAQ,Medical and Precision Instruments,1988,44,2.26,105481371520
4,394800,3billion,KOSDAQ,Medical and Precision Instruments,5160,-200,-3.73,163489491600


In [7]:
# 코스피, 코스닥 데이터 합치기

krx_sector = pd.concat([sector_stk, sector_ksq]).reset_index(drop=True)     # 두 데이터 합쳐주고 reset_index()로 인덱스 리셋, drop=True로 인덱스로 세팅한 열을 삭제
krx_sector['종목명'] = krx_sector['종목명'].str.strip()     # 종목명에 공백이 있는 경우 삭제
krx_sector['기준일'] = biz_day

krx_sector.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일
0,244920,A Plus Asset,KOSPI,Other Financial Companies,4260,-135,-3.07,96308772180,20250108
1,140910,A REIT,KOSPI,Real Estate,3130,0,0.00,13969290160,20250108
2,015260,A&P,KOSPI,Electrical Equipment & Electronics,613,6,0.99,27656656022,20250108
3,078520,ABLE C&C,KOSPI,Chemicals,7250,-60,-0.82,188602667250,20250108
4,018250,AEKYUNG INDUSTRIAL,KOSPI,Chemicals,12930,0,0.00,341480459550,20250108


## 10.2.2 개별종목 지표 크롤링

In [8]:
# 개별지표 OTP 생성 후 데이터 불러오기

import cloudscraper     # 보안서비스 우회 요청 처리 라이브러리
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'searchType':'1',
    'mktId':'ALL',
    'trdDd':biz_day,
    'csvxls_isNo':'false',
    'name':'fileDown',
    'url':'dbms/MDC/STAT/standard/MDCSTAT03501'
}
# headers={'Referer':'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
scraper = cloudscraper.create_scraper()
otp = scraper.post(gen_otp_url, params=gen_otp_data).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
krx_ind = scraper.post(down_url, params={'code': otp})

krx_ind = pd.read_csv(BytesIO(krx_ind.content), encoding='EUC-KR')
krx_ind['종목명'] = krx_ind['종목명'].str.strip()   # 종목명 없는 항목은 삭제
krx_ind['기준일'] = biz_day

krx_ind.head()

,종목코드,종목명,종가,대비,등락률,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률,기준일
0,060310,3S,1988,44,2.26,54.0,36.81,NaN,NaN,998.0,1.99,0,0.00,20250108
1,095570,AJ네트웍스,4245,-30,-0.70,367.0,11.57,1301.0,3.26,9326.0,0.46,270,6.36,20250108
2,006840,AK홀딩스,9950,-100,-1.00,2635.0,3.78,NaN,NaN,44339.0,0.22,200,2.01,20250108
3,054620,APS,5040,-110,-2.14,667.0,7.56,NaN,NaN,11683.0,0.43,0,0.00,20250108
4,265520,AP시스템,17320,640,3.84,3997.0,4.33,4607.0,3.76,21396.0,0.81,270,1.56,20250108


## 10.2.3 데이터 정리하기

In [9]:
diff = list(set(krx_sector['종목명']).symmetric_difference(set(krx_ind['종목명']))) # 대칭 차집합을 구하는 함수 / 대칭 차집합 : 두 집합의 합집합에서 공통 부분만 제외한 집합 - 한 집합에만 존재하는 요소들의 집합
print(diff)

['Biotoxtech', '유진스팩11호', 'JIN AIR', 'PonyLink', '엔브이에이치코리아', 'DongwonInd', 'Binex', 'KakaoBank', 'HUVIS', 'HYBE', 'ACTOZ SOFT', '카카오페이', '일양약품', 'K Car', '라온시큐어', 'Korea No.15 SPAC', '휴럼', '유진테크놀로지', '인크레더블버즈', 'KORCHIP', 'S-Oil(1P)', '씨유박스', '덕양산업', 'Tego', 'Chinyang Holdings', '정다운', '동원F&B', '삼영전자', 'HyundaiEng&Const', '한국철강', 'ICTK', 'HanmiPharm', 'KISCO Holdings', '아이디스', '듀오백', '라메디텍', '아바텍', 'MOALife', '우리넷', '효성티앤씨', 'HLB글로벌', '한국타이어앤테크놀로지', 'VISSEM', '한국콜마', 'Hansae', '한일현대시멘트', 'SK Securities No.13 SPAC', 'Studio Samick', 'SNT DYNAMICS', 'TELCOWARE', 'SECUCEN', 'CXI', '글로본', 'Macrogen', '한국첨단소재', '케이씨', 'CLS', 'HANDYSOFT', 'H.I.S.C(1P)', 'Sugentech', 'DL우', 'OSTEONIC', '유유제약1우', '한미글로벌', '한국제14호스팩', '알비더블유', 'SK바이오사이언스', '유안타제9호스팩', '엑셀세라퓨틱스', '플레이위드', '한세엠케이', 'S&K폴리텍', 'DANAL', '대성홀딩스', 'SK가스', 'DaedongGear', '오로라', 'HYUNDAIDEPTST', '피씨디렉트', 'DRB동일', 'Pharmsville', '이지바이오', '유유제약2우B', 'DI', 'LTC', 'SAMSUNG SPAC IX', '도이치모터스', 'CHOSUN WELDING', 'YM Tech', 'WOT', '경동도시가스', '

In [10]:
# pd.merge(): 두 개의 데이터프레임을 병합할 때 사용하는 함수, SQL의 JOIN 연산과 유사한 방식으로 데이터 결합
kor_ticker = pd.merge(krx_sector,   # 왼쪽 데이터프레임
                      krx_ind,      # 오른쪽 데이터프레임
                      on=krx_sector.columns.intersection(   # on="두 데이터프레임에서 공통으로 사용할 열 이름. 열 이름이 같을 경우에만 사용."
                          krx_ind.columns).tolist(),
                      how='outer')   # 병합 방식(outer:합집합)

kor_ticker.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률
0,000020,DongwhaPharm,KOSPI,Pharmaceuticals,6520,-10,-0.15,1.821132e+11,20250108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000020,동화약품,NaN,NaN,6520,-10,-0.15,NaN,20250108,991.0,6.58,NaN,NaN,13413.0,0.49,180.0,2.76
2,000040,KR MOTORS,KOSPI,Transportation Equipment & Components,482,-6,-1.23,2.898404e+10,20250108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000040,KR모터스,NaN,NaN,482,-6,-1.23,NaN,20250108,NaN,NaN,NaN,NaN,618.0,0.78,0.0,0.00
4,000050,Kyungbang,KOSPI,Distribution,6420,-100,-1.53,1.760060e+11,20250108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


일반적인 종목과 스팩, 우선주, 리츠, 기타 주식을 구분한다.

스팩(Special Purpose Acquisition Company, SPAC): 기업인수를 목적으로 하는 페이퍼컴퍼니
    - 대부분 증권사 주관으로 설립되며, 스팩이 먼저 투자자들의 자금을 모아 주식 시장에 상장이 되고 나면, 그 이후에 괜찮은 비상장기업을 찾아 합병하는 방식으로 최종 기업 인수가 이루어짐

In [11]:
# pandas의 []: [] 내부의 조건이 True인 항목들을 뽑아낸다.

print(kor_ticker[kor_ticker['종목명'].str.contains('스팩|제[0-9]+호')]['종목명'].values)

['미래에셋비전스팩1호' '키움제6호스팩' '신한제10호스팩' '엔에이치스팩23호' '케이비제21호스팩' '삼성스팩6호'
 '신영스팩8호' '유안타제9호스팩' '키움제7호스팩' '유안타제10호스팩' '하나금융25호스팩' '에스케이증권제8호스팩'
 '엔에이치스팩24호' '엔에이치스팩25호' '삼성스팩7호' '엔에이치스팩26호' 'IBKS제20호스팩' '교보13호스팩'
 '엔에이치스팩27호' '유진스팩9호' '대신밸런스제14호스팩' 'IBKS제21호스팩' '미래에셋드림스팩1호' '유안타제11호스팩'
 '비엔케이제1호스팩' '신영스팩9호' '유안타제12호스팩' '미래에셋비전스팩2호' '하나26호스팩' '키움제8호스팩'
 '하나27호스팩' '삼성스팩8호' 'IBKS제22호스팩' '미래에셋비전스팩3호' '유안타제13호스팩' '하이제8호스팩'
 '유안타제14호스팩' '엔에이치스팩29호' '상상인제4호스팩' '신한제11호스팩' '하나29호스팩' '하나28호스팩'
 'KB제25호스팩' '한화플러스제4호스팩' '에스케이증권제9호스팩' 'DB금융스팩11호' '교보14호스팩' '대신밸런스제15호스팩'
 '대신밸런스제16호스팩' '에스케이증권제10호스팩' 'KB제26호스팩' '한국제12호스팩' '에이치엠씨제6호스팩'
 '한국제13호스팩' 'KB제27호스팩' '교보15호스팩' '엔에이치스팩30호' 'IBKS제23호스팩' '삼성스팩9호'
 '유진스팩10호' 'IBKS제24호스팩' '하나30호스팩' '하나31호스팩' '대신밸런스제17호스팩' '신영스팩10호'
 '에스케이증권제11호스팩' '에스케이증권제12호스팩' '유안타제15호스팩' '비엔케이제2호스팩' '에스케이증권제13호스팩'
 '유안타제16호스팩' '신한제12호스팩' '신한제13호스팩' '하나32호스팩' '하나33호스팩' 'KB제28호스팩'
 '에이치엠씨제7호스팩' '미래에셋비전스팩4호' '미래에셋비전스팩5호' '한국제14호스팩' 'DB금융스팩12호' '이베스트스팩6호'
 'KB제29호스팩' '미래에셋비전스팩6호' '대신밸런스제1

In [12]:
# 종목코드 끝이 0이 아닌 종목은 우선주에 해당, 우선주를 뽑는 코드

print(kor_ticker[kor_ticker['종목코드'].str[-1:] != '0']['종목명'].values)

['SAMYANGHOLDINGS(1P)' '삼양홀딩스우' 'HITEJINRO(2PB)' '하이트진로2우B' 'Yuhan(1P)'
 '유한양행우' 'HITEJINROHOLDINGS(1P)' '하이트진로홀딩스우' 'DOOSAN(1P)' '두산우'
 'DOOSAN(2PB)' '두산2우B' 'DL(1P)' 'DL우' 'YuyuPharma(1P)' '유유제약1우'
 'YuyuPharma(2PB)' '유유제약2우B' 'NorooHoldings(1P)' '노루홀딩스우'
 'HeungkukF&MIns(1P)' '흥국화재우' 'HyundaiEng&Con(1P)' '현대건설우'
 'SamsungF&MIns(1P)' '삼성화재우' 'Hanwha(1P)' '한화우' 'HANWHA(3PB)' '한화3우B'
 'CJ(1P)' 'CJ우' 'CJ(4PC)' 'CJ4우(전환)' 'JWPHARMA(1P)' 'JW중외제약우'
 'JWPHARMA(2PB)' 'JW중외제약2우B' 'BookookSecu(1P)' '부국증권우' 'BYC(1P)' 'BYC우'
 'SKSecu(1P)' 'SK증권우' 'TONGYANG(1P)' '동양우' 'TONGYANG(2PB)' '동양2우B'
 'Daesang(1P)' '대상우' 'HanyangSecu(1P)' '한양증권우' 'TS(1P)' '대한제당우'
 'KOLON CORP(1P)' '코오롱우' 'NEXENTIRE(1PB)' '넥센타이어1우B' 'CHINHUNG INT(1PB)'
 '진흥기업우B' 'CHINHUNG INT (2PB)' '진흥기업2우B' 'AmoreG(1P)' '아모레G우'
 'AMOREG(3PC)' '아모레G3우(전환)' 'KUMHOE&C(1P)' '금호건설우' 'KolonGlobal(1P)'
 '코오롱글로벌우' 'YuhwaSecu(1P)' '유화증권우' 'Yuanta Securities(1P)' '유안타증권우'
 'KAL(1P)' '대한항공우' 'H.I.S.C(1P)' '한화투자증권우' 'DaishinSecu(1P)' '대신증권우'
 'DAISH

In [13]:
# 리츠 종목은 종목명이 리츠로 끝남

print(kor_ticker[kor_ticker['종목명'].str.endswith('리츠')]['종목명'].values)

[]


In [14]:
import numpy as np

# np.where(condition, val_if_true, val_if_false): 조건이 참이면 val_if_true를, 거짓이면 val_if_false를 반환
# 거짓에 np.where를 중첩으로 넣는 경우 - 거짓일 경우에 중첩된 np.where()을 실행

kor_ticker['종목구분'] = np.where(kor_ticker['종목명'].str.contains('스팩|재[0-9]+호'), '스팩',
                            np.where(kor_ticker['종목코드'].str[-1:] != '0', '우선주',
                                     np.where(kor_ticker['종목명'].str.endswith('리츠'), '리츠',
                                              np.where(kor_ticker['종목명'].isin(diff), '기타',
                                                  '보통주'))))

# df.reset_index(): 데이터프레임의 인덱스를 초기화하거나 재설정하는 데 사용됨
# 기존 인덱스를 기본 정수형 인덱스로 변환하고, 변환되기 전 인덱스를 데이터프레임의 열로 변환하거나 제거할 수 있음
kor_ticker = kor_ticker.reset_index(drop=True)
# 열 이름의 공백 삭제
kor_ticker.columns = kor_ticker.columns.str.replace(' ', '')
kor_ticker = kor_ticker[['종목코드', '종목명', '시장구분', '종가',
                         '시가총액', '기준일', 'EPS', '선행EPS', 'BPS', '주당배당금', '종목구분']]
# SQL에는 NaN이 입력되지 않으므로 None으로 재표기
kor_ticker = kor_ticker.replace({np.nan: None})
# 기준일을 datetime 형태로 변경한다
kor_ticker['기준일'] = pd.to_datetime(kor_ticker['기준일'])

kor_ticker.head()

,종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분
0,000020,DongwhaPharm,KOSPI,6520,182113184400.0,2025-01-08,None,None,None,None,기타
1,000020,동화약품,None,6520,None,2025-01-08,991.0,None,13413.0,180.0,기타
2,000040,KR MOTORS,KOSPI,482,28984042376.0,2025-01-08,None,None,None,None,기타
3,000040,KR모터스,None,482,None,2025-01-08,None,None,618.0,0.0,기타
4,000050,Kyungbang,KOSPI,6420,176006033400.0,2025-01-08,None,None,None,None,기타


In [15]:
# 다운로드받은 정보를 kor_ticker 테이블에 upsert 형태로 저장

import pymysql

con = pymysql.connect(user='root',
                      passwd='1234',
                      host='127.0.0.1',
                      db='stock_db',
                      charset='utf8')

mycursor = con.cursor()

query = f"""
    insert into kor_ticker (종목코드, 종목명, 시장구분, 종가, 시가총액, 기준일, EPS, 선행EPS, BPS, 주당배당금, 종목구분)
    values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) as new
    on duplicate key update
    종목명=new.종목명, 시장구분=new.시장구분, 종가=new.종가, 시가총액=new.시가총액, EPS=new.EPS, 선행EPS=new.선행EPS,
    BPS=new.BPS, 주당배당금=new.주당배당금, 종목구분=new.종목구분;
"""

args = kor_ticker.values.tolist()

mycursor.executemany(query, args)   # cursor.executemany(query, args): 같은 SQL문을 여러번 반복적으로 실행할 때 사용 / args는 삽입할 다수의 데이터셋의 모임


con.commit()    # 데이터베이스 변경사항을 실제 db에 저장
con.close()     # 데이터베이스와의 연결을 종료